# STEM Algorithm

In [1]:
import numpy as np
import pyomo.environ as pyo

In [2]:
ITERATIONS = 3
I, J = 3, 4

Cij = [
    [10, 30, 50, 100],
    [1, 1, 0, 0],
    [1, 4, 6, 2]
]
Aij = [
    [5, 3, 2, 0],
    [0, 0, 3, 8],
    [2, 3, 4, 6]
]
Bi = [240, 320, 180]

In [3]:
f = lambda i, *x: np.dot(x, np.array(Cij)[i].reshape(-1, 1))

In [4]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(0, I-1)
model.j = pyo.RangeSet(0, J-1)

model.a = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Aij)))
model.b = pyo.Param(model.i, initialize=dict(enumerate(Bi)))
model.c = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Cij)))

model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)


def cons_rule(model, i):
    return sum(model.a[i, j]*model.x[j] for j in model.j) <= model.b[i]
model.cons = pyo.Constraint(model.i, rule=cons_rule)

In [5]:
income_matrix = np.empty((I, I))

In [6]:
for i in range(I):
    model.obj = pyo.Objective(
        expr=sum(model.c[i, j]*model.x[j] for j in model.j),
        sense=pyo.maximize
    )

    opt = pyo.SolverFactory('cplex')
    opt.solve(model)

    x = [pyo.value(model.x[j]) for j in model.j]
    obj = pyo.value(model.obj)

    del model.obj

    matrix = np.array([])
    for j in range(I):
        if i == j:
            income_matrix[i, j] = obj
            continue  
        income_matrix[i, j] = f(j, *x)

In [28]:
alpha = np.empty(I)

In [29]:
for i in range(I):
    f_star = income_matrix[i, i]
    f_min = min(income_matrix[:, i])
    c = np.floor(np.sqrt(sum(np.array(Cij)[i])))

    if income_matrix[i, i] > 0:
        alpha[i] = (f_star-f_min)/(f_star*c)
    else:
        alpha[i] = (f_min-f_star)/(f_min*c)


In [31]:
pi = alpha/sum(alpha)

In [34]:
model.lambd = pyo.Var(domain=pyo.NonNegativeReals)
model.obj = pyo.Objective(expr=model.lambd)

model.cons_dm = pyo.ConstraintList()
for i in model.i:
    model.cons_dm.add(
        expr=model.lambd>=(income_matrix[i, i]-sum(model.c[i, j]*model.x[j] for j in model.j))*pi[i]
    )

In [36]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(i, *x) for i in range(I)]

In [37]:
res

[array([2223.88059701]), array([38.80597015]), array([176.41791045])]